In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [ ]:
spark = (SparkSession
 .builder
 .appName("City_State")
 .getOrCreate())

In [ ]:
schema = StructType([
             StructField('City', StringType(), True),
             StructField('State', StringType(), True)
         ])

In [ ]:
df =  spark.read.option('header','True').schema(schema).option('sep',',').csv('German_Cities_And_State.csv')

In [ ]:
df = df.dropDuplicates()

In [ ]:
"""
Assigning a row number to them, so that it can be referred later in the model

"""

window = Window.orderBy('State')
df = df.withColumn('Id',row_number().over(window))

df.show(truncate=False)

In [ ]:
"""

Will create a State folder and have this cleaned and tranformed data there

"""

df.write.format('csv').option('header','true').mode('overwrite').save('State')